# 线性回归的简洁实现
在过去的几年里，出于对深度学习强烈的兴趣， 许多公司、学者和业余爱好者开发了各种成熟的开源框架。 这些框架可以自动化基于梯度的学习算法中重复性的工作。 在 3.2节中，我们只运用了： （1）通过张量来进行数据存储和线性代数； （2）通过自动微分来计算梯度。 实际上，由于数据迭代器、损失函数、优化器和神经网络层很常用， 现代深度学习库也为我们实现了这些组件。

本节将介绍如何通过使用深度学习框架来简洁地实现 3.2节中的线性回归模型。就不需要我们手写了。

In [1]:
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

2023-11-27 17:15:29.884671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-27 17:15:29.884712: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 生成数据集

In [2]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)
print("y = ", features)
print("x = ", labels)

y =  tf.Tensor(
[[ 0.11894727  0.24298263]
 [ 1.2870909   1.9468771 ]
 [ 0.05740219  0.4054549 ]
 ...
 [ 0.15175322  0.28111246]
 [-1.4848349  -0.52007747]
 [-0.88650125  0.16432466]], shape=(1000, 2), dtype=float32)
x =  tf.Tensor(
[[ 3.59882045e+00]
 [ 1.67489320e-01]
 [ 2.92667389e+00]
 [ 8.83394051e+00]
 [ 4.23736858e+00]
 [ 4.94903755e+00]
 [-1.60571679e-01]
 [ 6.95879078e+00]
 [ 5.67643976e+00]
 [ 1.00073271e+01]
 [ 9.58525276e+00]
 [ 2.91168189e+00]
 [ 4.74333906e+00]
 [ 4.81058264e+00]
 [-5.32903429e-03]
 [-7.95379877e+00]
 [ 1.08190894e+00]
 [ 1.00671053e+01]
 [ 4.78741169e+00]
 [ 4.49038839e+00]
 [ 8.65771770e+00]
 [ 7.61070395e+00]
 [ 9.51909161e+00]
 [ 3.01051784e+00]
 [ 9.33925533e+00]
 [ 4.62630701e+00]
 [ 3.40852809e+00]
 [-8.10118914e-01]
 [ 1.02665796e+01]
 [ 1.81626236e+00]
 [ 2.30591846e+00]
 [ 7.31086254e+00]
 [ 5.33202457e+00]
 [ 5.64963913e+00]
 [ 3.13504815e+00]
 [ 3.18401790e+00]
 [ 1.00096350e+01]
 [ 9.49145889e+00]
 [ 4.67967629e-01]
 [ 5.24252224e+00]
 [ 6.27

2023-11-27 17:15:31.252543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-27 17:15:31.252709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-27 17:15:31.252756: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-11-27 17:15:31.252784: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-11-27 17:15:31.252812: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

# 读取数据集

In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个TensorFlow数据迭代器"""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.57635796, -0.37467742],
        [-0.5267973 ,  0.4111021 ],
        [ 0.56866837, -0.22415335],
        [ 0.7091218 , -2.748743  ],
        [ 0.38084972,  0.49896145],
        [ 0.903892  ,  0.93667895],
        [ 0.03687494, -2.2524283 ],
        [ 1.2070277 ,  1.1229855 ],
        [-1.033653  , -1.2162797 ],
        [ 1.1031955 , -1.0571686 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 4.3291526],
        [ 1.7502929],
        [ 6.097267 ],
        [14.964193 ],
        [ 3.2562363],
        [ 2.8159397],
        [11.944191 ],
        [ 2.7881258],
        [ 6.2688046],
        [10.006953 ]], dtype=float32)>)

# 定义模型

In [4]:
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

# 初始化模型参数

In [5]:
initializers = tf.random_normal_initializer(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializers))

# 定义损失函数，优化算法

In [6]:
loss = tf.keras.losses.MeanSquaredError()
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

# 训练

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000264
epoch 2, loss 0.000102
epoch 3, loss 0.000102


In [8]:
w = net.get_weights()[0]
print('w的估计误差：', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('b的估计误差：', true_b - b)

w的估计误差： tf.Tensor([-5.1021576e-04  9.8705292e-05], shape=(2,), dtype=float32)
b的估计误差： [-2.9563904e-05]
